In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF


class MSPBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MSPBlock, self).__init__()

        # Pooling layers
        self.pool1 = torch.nn.AdaptiveAvgPool2d((32, 32))
        self.pool2 = torch.nn.AdaptiveAvgPool2d((16, 16))
        self.pool3 = torch.nn.AdaptiveAvgPool2d((8, 8))
        self.pool4 = torch.nn.AdaptiveAvgPool2d((4, 4))

        # 1x1 convolution layers
        self.conv1 = torch.nn.Conv2d(in_channels, out_channels // 16, kernel_size=1, stride=1, padding=0)
        self.conv2 = torch.nn.Conv2d(in_channels, out_channels // 8, kernel_size=1, stride=1, padding=0)
        self.conv3 = torch.nn.Conv2d(in_channels, out_channels // 4, kernel_size=1, stride=1, padding=0)
        self.conv4 = torch.nn.Conv2d(in_channels, out_channels // 2, kernel_size=1, stride=1, padding=0)

        # Up-sampling layers with corrected scale factors
        self.up1 = torch.nn.Upsample(scale_factor=(1, 1), mode='bilinear')
        self.up2 = torch.nn.Upsample(scale_factor=(2, 2), mode='bilinear')
        self.up3 = torch.nn.Upsample(scale_factor=(4, 4), mode='bilinear')
        self.up4 = torch.nn.Upsample(scale_factor=(8, 8), mode='bilinear')

    def forward(self, x):
        # Pooling
        pool1 = self.pool1(x)
        pool2 = self.pool2(x)
        pool3 = self.pool3(x)
        pool4 = self.pool4(x)

        # 1x1 convolution
        conv1 = self.conv1(pool1)
        conv2 = self.conv2(pool2)
        conv3 = self.conv3(pool3)
        conv4 = self.conv4(pool4)

        # Up-sampling
        up1 = self.up1(conv1)
        up2 = self.up2(conv2)
        up3 = self.up3(conv3)
        up4 = self.up4(conv4)

        # Concatenate
        output = torch.cat([x, up1, up2, up3, up4], dim=1)

        return output


class MSDPath(nn.Module):
    def __init__(self):
        super(MSDPath, self).__init__()
        self.encoder = nn.ModuleList()
        self.encoder.append(nn.Conv3d(1, 32, kernel_size=5, stride=1, padding=2))

        self.encoder.append(nn.Sequential(
            (nn.MaxPool3d(kernel_size=(2,2,1), stride=2)),
            (nn.Conv3d(32, 64, kernel_size=5, stride=1, padding=2))))

        self.encoder.append(nn.Sequential(
            (nn.MaxPool3d(kernel_size=(2,2,1), stride=2)),
            (nn.Conv3d(64, 128, kernel_size=5, stride=1, padding=2))))

        self.conv2 = nn.Conv2d(320, 16, kernel_size=1, stride=1)
        self.conv4 = nn.Conv2d(320, 32, kernel_size=1, stride=1)
        self.conv6 = nn.Conv2d(384, 64, kernel_size=1, stride=1)

    def forward(self, x):
        skips = []
        x = x.reshape(3, 1, 256, 256, 10)
        conv1 = self.encoder[0](x)

        conv3 = self.encoder[1](conv1)
        conv5 = self.encoder[2](conv3)


        r1 = conv1.view((3, 320, 256, 256))
        r2 = conv3.view((3, 320, 128, 128))
        r3 = conv5.view((3, 384, 64, 64))

        r1 = self.conv2(r1)
        r2 = self.conv4(r2)
        r3 = self.conv6(r3)

        skips.append(r3)
        skips.append(r2)
        skips.append(r1)

        print("Skips shape")
        for i in skips:
            print(i.shape)

        return skips

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)


class UNET(nn.Module):
    def __init__(self, in_channels=10, out_channels=1, features=[64, 128, 256]):
        super(UNET, self).__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=1, stride=2)

        # Down part

        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # UP part

        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(
                feature * 2, feature, kernel_size=2, stride=2
            ))
            self.ups.append(DoubleConv(feature * 2, feature))

        self.bottleneck = nn.Sequential(
            nn.Conv2d(features[-1], features[-1], kernel_size=3, stride=1, padding=1, bias=False),
            MSPBlock(features[-1], features[-1]),
            nn.Conv2d(sum(features)//2+features[-1]+16, features[-1]*2, kernel_size=3, stride=1, padding=1, bias=False)
        )
        # self.bottleneck = DoubleConv(features[-1], features[-1] * 2)

        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for index in range(0, len(self.ups), 2):
            x = self.ups[index](x)
            skip_connection = skip_connections[index // 2]

            # Resize x to match the shape of skip_connection
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[index + 1](concat_skip)

        return self.final_conv(x)


# def test():
#     x = torch.randn((1, 10, 256, 256))
#     model = UNET(in_channels=10, out_channels=1)
#     preds = model(x)
#     print()
#     print("The final shapes")
#     print(x.shape)
#     print(preds.shape)
#     assert preds.shape == x.shape

In [ ]:
!pip install rasterio

In [ ]:
import os
import rasterio
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from skimage.transform import resize

class PVDetectionDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, output_size=(256, 256)):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.output_size = output_size
        self.transform = transform
        self.image_files = os.listdir(image_dir)
        self.mask_files = os.listdir(mask_dir)

        # Check if the number of image files and mask files match
        # assert len(self.image_files) == len(self.mask_files), "Number of images and masks must match."

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])

        with rasterio.open(image_path) as src:
            image = src.read()
        mask = Image.open(mask_path)

        image = torch.tensor(image).permute(1,2,0).numpy()

        mask = mask.resize(self.output_size)
        mask_transform = transforms.Grayscale(1)
        mask = mask_transform(mask)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
import os
import torch
import torchvision
from torchvision import transforms
# from dataset import PVDetectionDataset, image_transform, mask_transform
from torch.utils.data import DataLoader
# from sklearn.metrics import matthews_corrcoef, cohen_kappa_score
import shutil



def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model):
    print("=> Loading Checkpoint")
    model.load_state_dict(checkpoint["state_dict"])


def get_loaders(batch_size, train_ratio, val_ratio):
    # Create a single dataset for all images in the folder
    image_folder = "images"
    masks_folder = "masks"
    test_image_folder = "test_images"
    test_masks_folder = "test_masks"

    seed = 42
    torch.manual_seed(seed)

    # Create the custom dataset using all images and annotations
    dataset = PVDetectionDataset(image_folder, masks_folder, transform=transform)
    final_test_dataset = PVDetectionDataset(test_image_folder, test_masks_folder, transform=transform)

    # Split the dataset into train, validation, and test sets if needed
    # Example of splitting into train, val, and test (adjust the split ratios):
    train_size = int(train_ratio * len(dataset))
    val_size = int(val_ratio * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

    # Create data loaders for training, validation, and testing
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    final_test_data_loader = torch.utils.data.DataLoader(final_test_dataset, batch_size=batch_size, shuffle=False)

    return train_data_loader, val_data_loader, test_data_loader, final_test_data_loader


def validate_and_check_accuracy(loader, model, loss_fn, device="cuda"):
    model.eval()
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    total_loss = 0

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            # Forward pass
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()  # Apply threshold for binary prediction
            val_loss = loss_fn(preds, y)
            total_loss += val_loss.item()

            num_correct += (preds == y).sum().item()
            num_pixels += torch.numel(y)

            # Calculate Dice score
            intersection = (preds * y).sum().item()
            union = preds.sum().item() + y.sum().item()
            dice_score += (2 * intersection) / (union + 1e-8)

    accuracy = (num_correct / num_pixels) * 100
    dice_score /= len(loader)
    average_loss = total_loss / len(loader)

    print(f"Got {num_correct}/{num_pixels} correct.")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Dice Score: {dice_score:.4f}")
    print("Validation Loss: {:.4f}".format(average_loss))

    model.train()

    return average_loss



def save_predictions_as_imgs(loader, model, folder="saved_images/", device="cuda"):
    model.eval()
    if os.path.exists(folder):
        shutil.rmtree(folder)  # Remove the folder and its contents
    os.makedirs(folder, exist_ok=True)  # Recreate the output folder

    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()

        # Save images
        torchvision.utils.save_image(preds, f"{folder}/pred_{idx}.png")
        torchvision.utils.save_image(y, f"{folder}/{idx}.png")

    model.train()


# Usage example:
# save_predictions_as_imgs(loader, model, folder="output_folder", device="cuda")


# Usage example:
# save_predictions_as_imgs(loader, model, folder="output_folder", device="cuda")

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn


def test_model(test_loader, model, device="cuda"):
    model.eval()  #
    total_correct = 0
    num_pixels = 0
    criterion = nn.BCEWithLogitsLoss()

    TP = 0
    TN = 0
    FP = 0
    FN = 0


    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc="Testing"):
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = torch.sigmoid(model(inputs))

            # Calculate binary predictions (assuming a threshold of 0.5)
            preds = (outputs > 0.5).float()
            num_pixels += torch.numel(targets)

            TP += ((preds == 1) & (targets == 1)).sum().item()
            TN += ((preds == 0) & (targets == 0)).sum().item()
            FP += ((preds == 1) & (targets == 0)).sum().item()
            FN += ((preds == 0) & (targets == 1)).sum().item()

            # You can also calculate Accuracy, Precision, Recall, and F1-score using these values:
        accuracy = (TP + TN) / num_pixels
        precision = TP / (TP + FP) if (TP + FP) != 0 else 0.0
        recall = TP / (TP + FN) if (TP + FN) != 0 else 0.0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0.0
        mcc = (TP * TN - FP * FN) / ((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))**0.5 if ((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) != 0 else 0.0

            # Calculate Kappa Coefficient
        total_agreement = (TP + TN) / num_pixels
        expected_agreement = ((TP + FP) * (TP + FN) + (TN + FP) * (TN + FN)) / (num_pixels * num_pixels)
        kappa = (total_agreement - expected_agreement) / (1 - expected_agreement) if (1 - expected_agreement) != 0 else 0.0


            # Print or use these values as needed.
        print("--------------------------------------")
        print("Evaluation Metrics\n")
        print(f"True Positives (TP): {TP}")
        print(f"True Negatives (TN): {TN}")
        print(f"False Positives (FP): {FP}")
        print(f"False Negatives (FN): {FN}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 Score: {f1_score}")
        print(f"Matthews Correlation Coefficient (MCC): {mcc}")
        print(f"Kappa Coefficient: {kappa}")

    # You can return the evaluation metrics as needed
    return accuracy, precision, recall, f1_score, mcc, kappa

In [ ]:
import torch
# import albumentations as A
# from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# HYPERPARAMETERS
LEARNING_RATE = 0.0001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
NUM_EPOCHS = 100
# NUM_WORKERS = 8
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
# PIN_MEMORY = True
LOAD_MODEL = False
CLIP_VALUE = 5


def train_fn(loader, model, optimizer, loss_fn):
    loop = tqdm(loader)
    total_loss = 0.0

    model.train()

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.to(device=DEVICE)

        optimizer.zero_grad()

        predictions = model(data)
        loss = loss_fn(predictions, targets)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # update from tqdm
        loop.set_postfix(loss=loss.item())

    average_loss = total_loss / len(loader)
    print(f"Training Loss: {average_loss:.4f}")


model = UNET(in_channels=10, out_channels=1).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

scheduler = lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,  # Reduce LR by half when the metric plateaus
    patience=2,  # Number of epochs with no improvement after which LR will be reduced
    verbose=True,  # Print LR updates
    min_lr=1e-8  # Minimum LR (optional)
)

# Define early stopping parameters
early_stopping_patience = 10  # Number of epochs with no improvement after which to stop training
best_val_loss = float('inf')  # Initialize with a large value

train_loader, val_loader, test_loader = get_loaders(
    BATCH_SIZE,
    TRAIN_RATIO,
    VAL_RATIO
)

if LOAD_MODEL:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)

for epoch in range(NUM_EPOCHS):
    train_fn(train_loader, model, optimizer, loss_fn)

    val_loss = validate_and_check_accuracy(val_loader, model, loss_fn, device=DEVICE)

    scheduler.step(val_loss)

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # Reset patience counter when there's improvement
        best_model = model
    else:
        patience_counter += 1

    # Early stopping check
    if patience_counter >= early_stopping_patience:
        print(f"Validation loss has not improved for {early_stopping_patience} epochs. Stopping early.")
        break

    # save model
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }


    save_checkpoint(checkpoint)

    # print some examples to a folder
    save_predictions_as_imgs(
        val_loader, best_model, folder="saved_images", device=DEVICE
    )

save_predictions_as_imgs(
    test_loader, best_model, folder="test_images", device=DEVICE
)

save_predictions_as_imgs(
    final_test_loader, best_model, folder="final_test_images", device=DEVICE
)


test_accuracy, precision, recall, f1_score, mcc, kappa = test_model(test_loader, best_model, device="cuda")

  0%|          | 0/12 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
100%|██████████| 12/12 [00:22<00:00,  1.84s/it, loss=0.591]


Training Loss: 0.7176
Got 879966/5308416 correct.
Accuracy: 16.58%
Dice Score: 0.3006
Validation Loss: 1.1360
=> Saving checkpoint


100%|██████████| 12/12 [00:20<00:00,  1.74s/it, loss=0.467]


Training Loss: 0.5127
Got 4312147/5308416 correct.
Accuracy: 81.23%
Dice Score: 0.0137
Validation Loss: 0.6962
=> Saving checkpoint


100%|██████████| 12/12 [00:21<00:00,  1.82s/it, loss=0.4]


...

Training Loss: 0.2577
Got 5164451/5308416 correct.
Accuracy: 97.29%
Dice Score: 0.9524
Validation Loss: 0.6354
Epoch 00040: reducing learning rate of group 0 to 7.8125e-07.
=> Saving checkpoint


100%|██████████| 12/12 [00:21<00:00,  1.83s/it, loss=0.258]


Training Loss: 0.2570
Got 5163894/5308416 correct.
Accuracy: 97.28%
Dice Score: 0.9521
Validation Loss: 0.6355
Validation loss has not improved for 10 epochs. Stopping early.


Testing: 100%|██████████| 3/3 [00:02<00:00,  1.42it/s]

--------------------------------------
Evaluation Metrics

True Positives (TP): 715676
True Negatives (TN): 4475528
False Positives (FP): 29452
False Negatives (FN): 18089
Accuracy: 0.9779195903260031
Precision: 0.9604739051545507
Recall: 0.9753476930624928
F1 Score: 0.9678536581077875
Matthews Correlation Coefficient (MCC): 0.9626093853567806
Kappa Coefficient: 0.915790983232865


### CHANGES MADE:

1) Added grad_norm_ for clipping gradients

2) Removed Scaler.

In [ ]:
# import torch
# from tqdm import tqdm
# import torch.nn as nn
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, cohen_kappa_score

# def test_model(test_loader, model, device="cuda"):
#     model.eval()  # Set the model to evaluation mode
#     total_correct = 0
#     total_samples = 0
#     criterion = nn.BCEWithLogitsLoss()  # You can change this loss function as needed

#     true_labels = []
#     predicted_labels = []

#     with torch.no_grad():
#         for inputs, targets in tqdm(test_loader, desc="Testing"):
#             inputs = inputs.to(device)
#             targets = targets.to(device)

#             # Forward pass
#             outputs = model(inputs)

#             # Compute loss (if needed)
#             loss = criterion(outputs, targets)

#             # Calculate binary predictions (assuming a threshold of 0.5)
#             predicted = (outputs > 0.5).float()

#             # Update lists for evaluation metrics
#             true_labels.extend(targets.cpu().numpy())
#             predicted_labels.extend(predicted.cpu().numpy())

#             total_samples += targets.size(0)
#             total_correct += (predicted == targets).sum().item()

#     # Calculate accuracy
#     accuracy = 100.0 * total_correct / total_samples
#     print(f"Test Accuracy: {accuracy:.2f}%")

#     # Calculate other evaluation metrics
#     precision = precision_score(true_labels, predicted_labels)
#     recall = recall_score(true_labels, predicted_labels)
#     f1 = f1_score(true_labels, predicted_labels)
#     mcc = matthews_corrcoef(true_labels, predicted_labels)
#     kappa = cohen_kappa_score(true_labels, predicted_labels)

#     print(f"Precision: {precision:.4f}")
#     print(f"Recall: {recall:.4f}")
#     print(f"F1-Score: {f1:.4f}")
#     print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
#     print(f"Kappa Coefficient: {kappa:.4f}")

#     # You can return the evaluation metrics as needed
#     return accuracy, precision, recall, f1, mcc, kappa

# # Usage:
# # Call the test_model function with your test data loader and trained model
# # test_accuracy, precision, recall, f1_score, mcc, kappa = test_model(test_loader, model, device="cuda")

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
# import gc
# gc.collect()

In [ ]:
# train_loader, val_loader, test_loader = get_loaders(
#         DATA_DIR,
#         BATCH_SIZE,
#         TRAIN_RATIO,
#         VAL_RATIO
# #         NUM_WORKERS,
# #         PIN_MEMORY,
#     )

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# for x, y in train_loader:
#     print("Input shape:", x.shape)
#     print("Label shape:", y.shape)
#     break  # Print only the first batch


In [ ]:
# train_dataset

In [ ]:
# !pip install rasterio

In [ ]:
torch.__version__

'2.1.0+cu118'